Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import cobra

from yeast8model import Yeast8Model

# Construct models of cells of interest

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
from yeast8model import biomass_component_list_orig

glucose_bounds = (-4.75, 0) # gives a sensible growth rate for wt
wt_y8 = Yeast8Model("./models/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
wt_y8.biomass_component_list = biomass_component_list_orig
wt_y8.model.reactions.r_1714.bounds = glucose_bounds
wt_y8.add_media_components(['r_1992'])

# Grid

Note: the grid thing should take ~8 minutes (probably varies: 5-10 minutes).

In [ ]:
from yeast8model import heatmap_ablation_grid, vget_ablation_ratio

Convenience functions

In [ ]:
@np.vectorize
def vget_growthrate(x):
    return x.ablated_flux[0]

In [ ]:
def grid_plots(
    ablation_result_array,
    exch_rate_dict,
    ratio_vmin=0.5,
    ratio_vmax=1.5,
    grid_xlabel_leader='Glucose exchange',
    grid_ylabel_leader='Ammonium exchange',
):
    # Generate numpy arrays from ablation_result_array
    ratio_array = vget_ablation_ratio(ablation_result_array)
    growthrate_array = vget_growthrate(ablation_result_array)
    growthrate_gradient = np.gradient(growthrate_array)
    growthrate_gradient_greater = np.abs(growthrate_gradient[0]) - np.abs(growthrate_gradient[1])
    
    # Prepare lists for ratio vs growthrate plot
    ratios = ratio_array[1:,1:].ravel()
    growthrates = growthrate_array[1:,1:].ravel()

    # Set up subplots
    fig, ax = plt.subplots(nrows=6, ncols=1, figsize=(7,36))
    
    # Set up axes parameters
    xmax = np.max(list(exch_rate_dict.values())[0])
    ymax = np.max(list(exch_rate_dict.values())[0])
    grid_xlabel = f"{grid_xlabel_leader} (% max = {xmax:.2f})"
    grid_ylabel = f"{grid_ylabel_leader} (% max = {ymax:.2f})"

    # Ratio
    heatmap_ablation_grid(
        ax[0],
        exch_rate_dict,
        ratio_array,
        percent_saturation=True,
        vmin=ratio_vmin,
        vmax=ratio_vmax,
    )
    ax[0].set_xlabel(grid_xlabel)
    ax[0].set_ylabel(grid_ylabel)
    ax[0].set_title("Ratio")

    # Growth rate
    heatmap_ablation_grid(
        ax[1], exch_rate_dict, growthrate_array, percent_saturation=True, cbar_label="growth rate"
    )
    ax[1].set_xlabel(grid_xlabel)
    ax[1].set_ylabel(grid_ylabel)
    ax[1].set_title("Growth rate")

    # Growth rate vs ratio
    ax[2].scatter(growthrates, ratios)
    ax[2].set_xlabel("Growth rate (/h)")
    ax[2].set_ylabel("Ablation ratio")
    ax[2].set_title("Growth rate vs ablation ratio")
    
    # C gradient
    heatmap_ablation_grid(
        ax[3],
        exch_rate_dict,
        growthrate_gradient[0],
        percent_saturation=True,
        vmin=None,
        vmax=None,
        center=0,
        cmap="PiYG",
    )
    ax[3].set_xlabel(grid_xlabel)
    ax[3].set_ylabel(grid_ylabel)
    ax[3].set_title(f"Gradient, {grid_xlabel_leader} axis")

    # N gradient
    heatmap_ablation_grid(
        ax[4],
        exch_rate_dict,
        growthrate_gradient[1],
        percent_saturation=True,
        vmin=None,
        vmax=None,
        center=0,
        cmap="PiYG",
    )
    ax[4].set_xlabel(grid_xlabel)
    ax[4].set_ylabel(grid_ylabel)
    ax[4].set_title(f"Gradient, {grid_ylabel_leader} axis")
    
    # Compare gradients from both axes
    heatmap_ablation_grid(
        ax[5],
        exch_rate_dict,
        growthrate_gradient_greater,
        percent_saturation=True,
        vmin=None,
        vmax=None,
        center=0,
        cmap="PuOr",
    )
    ax[5].set_xlabel(grid_xlabel)
    ax[5].set_ylabel(grid_ylabel)
    ax[5].set_title(
        "1 = change in glucose axis has greater magnitude\n0 = change in ammonium axis has greater magnitude"
    )

## Enzyme-constrained

`linspace` values are based on saturation exchange rates from the saturation curves.  These values may be different from the optimal uptake values from optimising the unmodified wild-type model.

### Glucose × ammonium

In [ ]:
exch_rate_dict = {
    "r_1714": np.linspace(0, 2*8.45, 32), # glucose
    "r_1654": np.linspace(0, 2*1.45, 32), # ammonium
}

In [ ]:
ablation_result_array = wt_ec.ablation_grid(exch_rate_dict)

In [ ]:
# Alternatively, load if saved
with open('ec_grid_glc_amm.pkl', 'rb') as handle:
    ablation_result_array = pickle.load(handle)

In [ ]:
# Dump data
with open('ec_grid_glc_amm.pkl', 'wb') as handle:
    pickle.dump(ablation_result_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict)

### Pyruvate × ammonium

In [ ]:
exch_rate_dict = {
    "r_2033": np.linspace(0, 2*4.27, 32), # pyruvate
    "r_1654": np.linspace(0, 2*1.45, 32), # ammonium
}

In [ ]:
ablation_result_array = wt_ec.ablation_grid(exch_rate_dict)

In [ ]:
# Alternatively, load if saved
with open('ec_grid_pyr_amm.pkl', 'rb') as handle:
    ablation_result_array = pickle.load(handle)

In [ ]:
# Dump data
with open('ec_grid_pyr_amm.pkl', 'wb') as handle:
    pickle.dump(ablation_result_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict, ratio_vmin=0.75, ratio_vmax=1.25, grid_xlabel_leader="Pyruvate exchange")

## Original yeast-GEM (no enzyme constraints)

### Glucose × ammonium

In [ ]:
exch_rate_dict = {
    "r_1714": np.linspace(0, 2*4.75, 32), # glucose
    "r_1654": np.linspace(0, 2*2.88, 32), # ammonium
}

In [ ]:
ablation_result_array = wt_y8.ablation_grid(exch_rate_dict)

In [ ]:
# Alternatively, load if saved
with open('y8_grid_glc_amm.pkl', 'rb') as handle:
    ablation_result_array = pickle.load(handle)

In [ ]:
# Dump data
with open('y8_grid_glc_amm.pkl', 'wb') as handle:
    pickle.dump(ablation_result_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict, ratio_vmin=0.75, ratio_vmax=1.25)

### Pyruvate × ammonium

In [ ]:
exch_rate_dict = {
    "r_2033": np.linspace(0, 2*13.32, 32), # pyruvate
    "r_1654": np.linspace(0, 2*2.88, 32), # ammonium
}

In [ ]:
ablation_result_array = wt_y8.ablation_grid(exch_rate_dict)

In [ ]:
# Alternatively, load if saved
with open('y8_grid_pyr_amm.pkl', 'rb') as handle:
    ablation_result_array = pickle.load(handle)

In [ ]:
# Dump data
with open('y8_grid_pyr_amm.pkl', 'wb') as handle:
    pickle.dump(ablation_result_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict, ratio_vmin=0.75, ratio_vmax=1.25, grid_xlabel_leader="Pyruvate exchange")